In [1]:
import pandas as pd
import numpy as np

In [3]:
data=pd.read_csv('머신러닝 전처리 데이터.CSV',encoding='EUC-KR')

In [19]:
data

,시군구,단지명,전용면적(㎡),계약년월,계약일,계약날짜,층,건축나이,거래건수,기준금리,법정동코드,지하철역수,거래금액(만원)
0,광주광역시 광산구 도산동,대덕1,59.3100,202303,18,2023-03-18,4,33,20,3.5,2920010200,1,9800
1,광주광역시 광산구 도산동,대주피오레1차,84.9800,202302,7,2023-02-07,2,23,14,3.5,2920010200,1,21000
2,광주광역시 광산구 도산동,대주피오레1차,84.9800,202305,31,2023-05-31,5,23,16,3.5,2920010200,1,22350
3,광주광역시 광산구 도산동,대주피오레2차,84.7575,202305,9,2023-05-09,5,22,16,3.5,2920010200,1,22400
4,광주광역시 광산구 도산동,대주피오레2차,84.7575,202305,13,2023-05-13,13,22,16,3.5,2920010200,1,22000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
425074,광주광역시 서구 화정동,화정우미,115.1850,200611,10,2006-11-10,17,10,131,4.5,2914011900,1,19000
425075,광주광역시 서구 화정동,화정우미,84.8300,200611,27,2006-11-27,6,10,131,4.5,2914011900,1,12000
425076,광주광역시 서구 화정동,화정우미,59.8300,200611,30,2006-11-30,2,10,131,4.5,2914011900,1,7700
425077,광주광역시 서구 화정동,화정우미,84.8400,200612,6,2006-12-06,16,10,124,4.5,2914011900,1,12170


In [3]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install lightgbm

                                              0.0/1.3 MB ? eta -:--:--
     ----------                               0.4/1.3 MB 7.4 MB/s eta 0:00:01
     ------------------                       0.6/1.3 MB 6.4 MB/s eta 0:00:01
     ----------------------------------       1.1/1.3 MB 8.8 MB/s eta 0:00:01
     ---------------------------------------- 1.3/1.3 MB 8.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install tqdm

                                              0.0/77.1 kB ? eta -:--:--
     ---------------------------------------- 77.1/77.1 kB 2.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [14]:
from sklearn.linear_model import LinearRegression, ElasticNet, Lasso, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import  LabelEncoder
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score
from tqdm import tqdm
import xgboost as xgb
import re

In [9]:
import lightgbm as lgb


OSError: [WinError 8] 메모리 리소스가 부족하여 이 명령을 처리할 수 없습니다

In [13]:
data.head(2)

,시군구,단지명,전용면적(㎡),계약년월,계약일,계약날짜,층,건축나이,거래건수,기준금리,법정동코드,지하철역수,거래금액(만원)
0,광주광역시 광산구 도산동,대덕1,59.31,202303,18,2023-03-18,4,33,20,3.5,2920010200,1,9800
1,광주광역시 광산구 도산동,대주피오레1차,84.98,202302,7,2023-02-07,2,23,14,3.5,2920010200,1,21000


In [5]:
data[data['시군구']=='광주광역시 남구 봉선동']

,시군구,단지명,전용면적(㎡),계약년월,계약일,계약날짜,층,건축나이,거래건수,기준금리,법정동코드,지하철역수,거래금액(만원)
2104,광주광역시 남구 봉선동,금호2,122.8100,202301,12,2023-01-12,15,28,29,3.5,2915511500,0,64000
2105,광주광역시 남구 봉선동,금호2,122.8100,202301,15,2023-01-15,1,28,29,3.5,2915511500,0,60500
2106,광주광역시 남구 봉선동,금호2,84.9600,202301,21,2023-01-21,6,28,29,3.5,2915511500,0,49500
2107,광주광역시 남구 봉선동,금호2,84.8300,202303,25,2023-03-25,6,28,27,3.5,2915511500,0,49500
2108,광주광역시 남구 봉선동,금호2,84.8300,202304,18,2023-04-18,13,28,27,3.5,2915511500,0,49000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
406940,광주광역시 남구 봉선동,포스코더샵,59.8900,200612,23,2006-12-23,5,2,100,4.5,2915511500,0,13650
406941,광주광역시 남구 봉선동,포스코더샵,59.8900,200612,27,2006-12-27,17,2,100,4.5,2915511500,0,13000
406942,광주광역시 남구 봉선동,포스코더샵,84.3200,200612,27,2006-12-27,8,2,100,4.5,2915511500,0,23900
406943,광주광역시 남구 봉선동,포스코더샵,150.9700,200612,27,2006-12-27,2,2,100,4.5,2915511500,0,41000


In [6]:
data['시군구']

0         광주광역시 광산구 도산동
1         광주광역시 광산구 도산동
2         광주광역시 광산구 도산동
3         광주광역시 광산구 도산동
4         광주광역시 광산구 도산동
              ...      
425074     광주광역시 서구 화정동
425075     광주광역시 서구 화정동
425076     광주광역시 서구 화정동
425077     광주광역시 서구 화정동
425078     광주광역시 서구 화정동
Name: 시군구, Length: 425079, dtype: object

In [15]:
X = data[['전용면적(㎡)', '계약년월','층', '건축나이','거래건수','기준금리','법정동코드','지하철역수']]
y = data['거래금액(만원)']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
# dacon 하이퍼파라미터튜닝 모델. 연습용
lrmodel = LinearRegression(n_jobs=-1)
ridge = Ridge(alpha=0.8, random_state=1)
lasso = Lasso(alpha = 0.01, random_state=1)
enet = ElasticNet(alpha=0.03, l1_ratio=0.01, random_state=1)
d_tree = DecisionTreeRegressor(max_depth=6, min_samples_split=10, min_samples_leaf=15, random_state=1)
rf = RandomForestRegressor(n_estimators=100,  max_depth=9, min_samples_split=50,
                           min_samples_leaf=5, random_state=1, n_jobs=-1)
model_xgb = xgb.XGBRegressor(n_estimators=100, max_depth=9, min_child_weight=5, gamma=0.1, n_jobs=-1 )
model_lgb = lgb.LGBMRegressor(n_estimators=100, max_depth=9, min_child_weight=5, n_jobs=-1,num_leaves=20)

In [16]:
# 특성없음
lrmodel = LinearRegression()
ridge = Ridge()
lasso = Lasso()
enet = ElasticNet()
d_tree = DecisionTreeRegressor()
rf = RandomForestRegressor()
model_xgb = xgb.XGBRegressor()
# model_lgb = lgb.LGBMRegressor()

In [17]:
def print_rmse_r2_score(model):
    # cv별 프린팅, 평균 저장
    model_name, rmse_list, r2_list = rmse_r2_cv(model)
    for i, (rmse, r2) in enumerate(zip(rmse_list, r2_list), start=1):
         print(f'{i} FOLDS: {model_name} RMSE: {rmse:.4f}, R2: {r2:.4f}')
    print(f'\n{model_name} mean RMSE: {np.mean(rmse_list):.4f}, mean R2: {np.mean(r2_list):.4f}')
    print('='*40)
    return model_name, np.mean(rmse_list), np.mean(r2_list)

def rmse_r2_cv(model):
    # cv별로 학습하는 함수
    tscv = TimeSeriesSplit(n_splits=10)
    rmse_list = []
    r2_list = []
    model_name = model.__class__.__name__
    for _, (train_index, test_index) in tqdm(enumerate(tscv.split(X), start=1), desc=f'{model_name} Cross Validations...', total=10):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        clf = model.fit(X_train, y_train)
        pred = clf.predict(X_test)
        rmse = RMSE(y_test, pred)
        r2 = r2_score(y_test, pred)
        rmse_list.append(rmse)
        r2_list.append(r2)
    return model_name, rmse_list, r2_list

def RMSE(y, y_pred):
    rmse = mean_squared_error(y, y_pred) ** 0.5
    return rmse

In [19]:
models = []
rmse_scores = []
r2_scores = []
for model in [lrmodel, ridge, lasso, enet, d_tree, model_xgb]:
    model_name, mean_rmse_score, mean_r2_score = print_rmse_r2_score(model)
    models.append(model_name)
    rmse_scores.append(mean_rmse_score)
    r2_scores.append(mean_r2_score)

LinearRegression Cross Validations...: 100%|██████████| 10/10 [00:00<00:00, 20.35it/s]


1 FOLDS: LinearRegression RMSE: 10084.0162, R2: 0.6496
2 FOLDS: LinearRegression RMSE: 9080.6999, R2: 0.5288
3 FOLDS: LinearRegression RMSE: 6591.1190, R2: 0.6617
4 FOLDS: LinearRegression RMSE: 5906.4291, R2: 0.5981
5 FOLDS: LinearRegression RMSE: 5854.3037, R2: 0.4538
6 FOLDS: LinearRegression RMSE: 5383.9206, R2: 0.5477
7 FOLDS: LinearRegression RMSE: 5841.3210, R2: 0.5347
8 FOLDS: LinearRegression RMSE: 8988.2098, R2: -0.3882
9 FOLDS: LinearRegression RMSE: 4756.1851, R2: 0.2545
10 FOLDS: LinearRegression RMSE: 4739.9987, R2: 0.0180

LinearRegression mean RMSE: 6722.6203, mean R2: 0.3859


Ridge Cross Validations...: 100%|██████████| 10/10 [00:00<00:00, 31.13it/s]


1 FOLDS: Ridge RMSE: 10084.0369, R2: 0.6496
2 FOLDS: Ridge RMSE: 9080.6665, R2: 0.5288
3 FOLDS: Ridge RMSE: 6591.1186, R2: 0.6617
4 FOLDS: Ridge RMSE: 5906.4287, R2: 0.5981
5 FOLDS: Ridge RMSE: 5854.3037, R2: 0.4538
6 FOLDS: Ridge RMSE: 5383.9201, R2: 0.5478
7 FOLDS: Ridge RMSE: 5841.3208, R2: 0.5347
8 FOLDS: Ridge RMSE: 8988.2091, R2: -0.3882
9 FOLDS: Ridge RMSE: 4756.1838, R2: 0.2545
10 FOLDS: Ridge RMSE: 4739.9990, R2: 0.0180

Ridge mean RMSE: 6722.6187, mean R2: 0.3859


Lasso Cross Validations...: 100%|██████████| 10/10 [00:00<00:00, 15.95it/s]


...

1 FOLDS: ElasticNet RMSE: 10187.8392, R2: 0.6423
2 FOLDS: ElasticNet RMSE: 8660.3586, R2: 0.5714
3 FOLDS: ElasticNet RMSE: 6574.6227, R2: 0.6634
4 FOLDS: ElasticNet RMSE: 5887.6433, R2: 0.6007
5 FOLDS: ElasticNet RMSE: 5846.1615, R2: 0.4553
6 FOLDS: ElasticNet RMSE: 5350.0822, R2: 0.5534
7 FOLDS: ElasticNet RMSE: 5822.6967, R2: 0.5377
8 FOLDS: ElasticNet RMSE: 8951.1368, R2: -0.3768
9 FOLDS: ElasticNet RMSE: 4650.9873, R2: 0.2871
10 FOLDS: ElasticNet RMSE: 4763.4061, R2: 0.0083

ElasticNet mean RMSE: 6669.4935, mean R2: 0.3943


DecisionTreeRegressor Cross Validations...: 100%|██████████| 10/10 [00:16<00:00,  1.65s/it]


1 FOLDS: DecisionTreeRegressor RMSE: 11789.9074, R2: 0.5210
2 FOLDS: DecisionTreeRegressor RMSE: 7443.6819, R2: 0.6834
3 FOLDS: DecisionTreeRegressor RMSE: 5646.9410, R2: 0.7517
4 FOLDS: DecisionTreeRegressor RMSE: 4750.2557, R2: 0.7400
5 FOLDS: DecisionTreeRegressor RMSE: 3770.4280, R2: 0.7734
6 FOLDS: DecisionTreeRegressor RMSE: 3357.0146, R2: 0.8242
7 FOLDS: DecisionTreeRegressor RMSE: 4517.7316, R2: 0.7217
8 FOLDS: DecisionTreeRegressor RMSE: 3781.9967, R2: 0.7542
9 FOLDS: DecisionTreeRegressor RMSE: 2610.0974, R2: 0.7755
10 FOLDS: DecisionTreeRegressor RMSE: 2371.3593, R2: 0.7542

DecisionTreeRegressor mean RMSE: 5003.9414, mean R2: 0.7299


XGBRegressor Cross Validations...: 100%|██████████| 10/10 [00:33<00:00,  3.32s/it]

1 FOLDS: XGBRegressor RMSE: 6539.0258, R2: 0.8527
2 FOLDS: XGBRegressor RMSE: 4717.6404, R2: 0.8728
3 FOLDS: XGBRegressor RMSE: 4358.4450, R2: 0.8521
4 FOLDS: XGBRegressor RMSE: 3444.6749, R2: 0.8633
5 FOLDS: XGBRegressor RMSE: 2780.2411, R2: 0.8768
6 FOLDS: XGBRegressor RMSE: 2651.7445, R2: 0.8903
7 FOLDS: XGBRegressor RMSE: 3251.0589, R2: 0.8559
8 FOLDS: XGBRegressor RMSE: 3385.2743, R2: 0.8031
9 FOLDS: XGBRegressor RMSE: 1750.6406, R2: 0.8990
10 FOLDS: XGBRegressor RMSE: 1635.4810, R2: 0.8831

XGBRegressor mean RMSE: 3451.4226, mean R2: 0.8649


In [21]:
result_df = pd.DataFrame({'Model': models, 'RMSE': rmse_scores, 'R2': r2_scores}).reset_index(drop=True)
result_df
# 하이퍼파라미터 튜닝 없는 자료

,Model,RMSE,R2
0,LinearRegression,6722.620306,0.385872
1,Ridge,6722.618718,0.385872
2,Lasso,6722.256191,0.385947
3,ElasticNet,6669.493452,0.394279
4,DecisionTreeRegressor,5003.941373,0.729934
5,XGBRegressor,3451.422637,0.864900


In [22]:
# 기존dacon 하이퍼파라미터 튜닝 자료 
result_df = pd.DataFrame({'Model': models, 'RMSE': rmse_scores, 'R2': r2_scores}).reset_index(drop=True)
result_df

,Model,RMSE,R2
0,LinearRegression,6722.620306,0.385872
1,Ridge,6722.618718,0.385872
2,Lasso,6722.256191,0.385947
3,ElasticNet,6669.493452,0.394279
4,DecisionTreeRegressor,5003.941373,0.729934
5,XGBRegressor,3451.422637,0.864900


In [24]:
X_predict = [[84, 202305,10,10 ,15,3.5,2920010200,0]]
X_predict_df= pd.DataFrame(X_predict,columns=['전용면적(㎡)', '계약년월','층', '건축나이','거래건수','기준금리','법정동코드','지하철역수'])

X = data[['전용면적(㎡)', '계약년월','층', '건축나이','거래건수','기준금리','법정동코드','지하철역수']]

In [25]:
print(f"LinearRegression: {lrmodel.predict(X_predict_df)[0]}")
print(f"Ridge: {ridge.predict(X_predict_df)[0]}")
print(f"Lasso: {lasso.predict(X_predict_df)[0]}")
print(f"ElasticNet: {enet.predict(X_predict_df)[0]}")
print(f"DecisionTreeRegressor: {d_tree.predict(X_predict_df)[0]}")
print(f"XGBRegressor: {model_xgb.predict(X_predict_df)[0]}")

LinearRegression: 36369.40774595458
Ridge: 36369.405329700094
Lasso: 36364.92155494355
ElasticNet: 36081.574030471966
DecisionTreeRegressor: 36000.0
XGBRegressor: 35704.38671875


In [22]:
lrmodel = LinearRegression()
ridge = Ridge()
lasso = Lasso()
enet = ElasticNet()
d_tree = DecisionTreeRegressor()
rf = RandomForestRegressor()
model_xgb = xgb.XGBRegressor()
model_lgb = lgb.LGBMRegressor()

In [ ]:
lrmodel = LinearRegression(n_jobs=-1)
ridge = Ridge(alpha=0.8, random_state=1)
lasso = Lasso(alpha = 0.01, random_state=1)
enet = ElasticNet(alpha=0.03, l1_ratio=0.01, random_state=1)
d_tree = DecisionTreeRegressor(max_depth=6, min_samples_split=10, min_samples_leaf=15, random_state=1)
rf = RandomForestRegressor(n_estimators=100,  max_depth=9, min_samples_split=50,
                           min_samples_leaf=5, random_state=1, n_jobs=-1)
model_xgb = xgb.XGBRegressor(n_estimators=100, max_depth=9, min_child_weight=5, gamma=0.1, n_jobs=-1 )
model_lgb = lgb.LGBMRegressor(n_estimators=100, max_depth=9, min_child_weight=5, n_jobs=-1,num_leaves=20)

##  하이퍼 파라미터 튜닝

In [28]:
from sklearn.model_selection import GridSearchCV

In [11]:
# Linear Regression
lr_params = {
    'fit_intercept': [True, False]
}
lr_grid_search = GridSearchCV(estimator=LinearRegression(n_jobs=-1),
                              param_grid=lr_params,
                              scoring='neg_mean_squared_error',
                              cv=10,
                              n_jobs=-1)
lr_grid_search.fit(X, y)
best_lr_model = lr_grid_search.best_estimator_

In [12]:


# Ridge
ridge_params = {
    'alpha': [0.1, 0.5, 1.0, 2.0]
}
ridge_grid_search = GridSearchCV(estimator=Ridge(random_state=1),
                                 param_grid=ridge_params,
                                 scoring='neg_mean_squared_error',
                                 cv=10,
                                 n_jobs=-1)
ridge_grid_search.fit(X, y)
best_ridge_model = ridge_grid_search.best_estimator_

In [13]:
# Lasso
lasso_params = {
    'alpha': [0.001, 0.01, 0.1, 1.0]
}
lasso_grid_search = GridSearchCV(estimator=Lasso(random_state=1),
                                 param_grid=lasso_params,
                                 scoring='neg_mean_squared_error',
                                 cv=10,
                                 n_jobs=-1)
lasso_grid_search.fit(X, y)
best_lasso_model = lasso_grid_search.best_estimator_

In [14]:
# ElasticNet
enet_params = {
    'alpha': [0.001, 0.01, 0.1, 1.0],
    'l1_ratio': [0.01, 0.1, 0.5, 0.9]
}
enet_grid_search = GridSearchCV(estimator=ElasticNet(random_state=1),
                                param_grid=enet_params,
                                scoring='neg_mean_squared_error',
                                cv=10,
                                n_jobs=-1)
enet_grid_search.fit(X, y)
best_enet_model = enet_grid_search.best_estimator_

In [15]:
# Decision Tree
d_tree_params = {
    'max_depth': [5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 5, 10]
}
d_tree_grid_search = GridSearchCV(estimator=DecisionTreeRegressor(random_state=1),
                                  param_grid=d_tree_params,
                                  scoring='neg_mean_squared_error',
                                  cv=10,
                                  n_jobs=-1)
d_tree_grid_search.fit(X, y)
best_d_tree_model = d_tree_grid_search.best_estimator_

In [ ]:
# RandomForest
rf_params = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 5, 10]
}
rf_grid_search = GridSearchCV(estimator=RandomForestRegressor(random_state=1, n_jobs=-1),
                              param_grid=rf_params,
                              scoring='neg_mean_squared_error',
                              cv=10,
                              n_jobs=-1)
rf_grid_search.fit(X, y)
best_rf_model = rf_grid_search.best_estimator_


In [30]:
# XGBoost
xgb_params = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 15],
    'min_child_weight': [1, 3, 5],
    'gamma': [0.1, 0.2, 0.3]
}
xgb_grid_search = GridSearchCV(estimator=xgb.XGBRegressor(n_jobs=-1),
                               param_grid=xgb_params,
                               scoring='neg_mean_squared_error',
                               cv=10,
                               n_jobs=-1)
xgb_grid_search.fit(X, y)
best_xgb_model = xgb_grid_search.best_estimator_


In [ ]:

# LightGBM
lgb_params = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 15],
    'min_child_weight': [1, 3, 5],
    'num_leaves': [10, 20, 30]
}
lgb_grid_search = GridSearchCV(estimator=lgb.LGBMRegressor(n_jobs=-1),
                               param_grid=lgb_params,
                               scoring='neg_mean_squared_error',
                               cv=10,
                               n_jobs=-1)
lgb_grid_search.fit(X, y)
best_lgb_model = lgb_grid_search.best_estimator_



In [31]:
# 출력
# print("Best Linear Regression Model:", best_lr_model)
# print("Best Ridge Model:", best_ridge_model)
# print("Best Lasso Model:", best_lasso_model)
# print("Best ElasticNet Model:", best_enet_model)
# print("Best Decision Tree Model:", best_d_tree_model)
print("Best XGBoost Model:", best_xgb_model)

Best XGBoost Model: XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.1, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=5, missing=nan, monotone_constraints=None,
             n_estimators=200, n_jobs=-1, num_parallel_tree=None,
             predictor=None, random_state=None, ...)


In [32]:
best_xgb_model # 아래 값 들을 넣어서 모델을 돌리면 가장 좋은 결과를 낼거야!

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.1, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=5, missing=nan, monotone_constraints=None,
             n_estimators=200, n_jobs=-1, num_parallel_tree=None,
             predictor=None, random_state=None, ...)